In [1]:
# load data
# overal 8092 images, trainset in 'Flickr_8k.trainImages.txt'
# overal 8092 images, validset in 'Flickr_8k.devImages.txt'
# overal 8092 images, testset in 'Flickr_8k.testImages'

In [84]:
import os
import pickle
import time
import numpy as np
from pprint import pprint
from IPython.display import SVG
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import corpus_bleu
from IPython.display import Image, display

In [3]:
def prepare_dataset(filename):
    image_ids = list()
    image_features = dict()
    captions = dict()
    
    # get image id
    with open(filename, 'r') as f:
        text = f.read()
        image_filenames = text.split('\n') # get image filenames in dataset: (train, valid, test)
        image_ids = [x.split('.')[0] for x in image_filenames if x] # remove '.jpg', igore empty filename

    # get image feature
    with open(os.getcwd() + '/image_features.pkl', 'rb') as f:
        all_features = pickle.load(f)
        image_features = { x: all_features[x] for x in image_ids }
        
    # get image caption
    with open(os.getcwd() + '/captions.txt', 'r') as f:
        text = f.read()
        lines = text.split('\n')
        for line in lines:
            space = line.index(' ')
            image_id = line[:space]
            caption = line[space+1:]
            
            # ignore empty caption
            if not caption:
                continue

            caption = 'startseq ' + caption + ' endseq' # add <start>, <end> token
            
            # append captions only in specified image_ids
            if image_id not in image_ids:
                continue

            if image_id in captions:
                captions[image_id].append(caption)
            else:
                captions[image_id] = [caption]
                    
    return image_ids, image_features, captions
        

In [4]:
        
# # get image id
# with open(os.getcwd() + '/Flickr8k_text/Flickr_8k.trainImages.txt', 'r') as f:
#     text = f.read()
#     filenames = text.split('\n')
#     train_image_ids = [x.split('.')[0] for x in filenames if x] # remove '.jpg', ignore empty line

# # get image feature
# with open(os.getcwd() + '/image_features.pkl', 'rb') as f:
#     all_features = pickle.load(f)
#     train_features = { x: all_features[x] for x in train_image_ids }
    
# # get image caption
# with open(os.getcwd() + '/captions.txt', 'r') as f:
#     text = f.read()
#     for line in text.split('\n'):
#         space_idx = line.index(' ')
#         image_id = line[:space_idx]
#         image_caption = line[space_idx+1:]
#         image_caption = ['startseq'] + image_caption.split() + ['endseq'] # add <start>, <end> token, keras tokenizer will remove '<'(punctuation)
#         image_caption = ' '.join(image_caption) # back to string
#         if image_id in train_image_ids:
#             if image_id in train_captions:
#                 train_captions[image_id].append(image_caption)
#             else:
#                 train_captions[image_id] = [image_caption]

In [5]:
# load each dataset

train_info_filename = os.getcwd() + '/Flickr8k_text/Flickr_8k.trainImages.txt'
valid_info_filename = os.getcwd() + '/Flickr8k_text/Flickr_8k.devImages.txt'
train_image_ids, train_feature_dict, train_caption_dict = prepare_dataset(train_info_filename)
valid_image_ids, valid_feature_dict, valid_caption_dict = prepare_dataset(valid_info_filename)

print('# of images in train set:', len(train_image_ids))
print('# of features in train set:', len(train_feature_dict)) # this should be same with above
print('# of images in train caption dict:', len(train_caption_dict))
print('# of images in valid set:', len(valid_image_ids))
print('# of features in valid set:', len(valid_feature_dict)) # this should be same with above
print('# of images in valid caption dict:', len(valid_caption_dict))

print('-------------------------------------------------------------------')
print('Exmaples in each variable')
print('train_image_ids[0]:', train_image_ids[0])
print('train_feature_dict[train_image_ids[0]].shape:', train_feature_dict[train_image_ids[0]].shape)
print('train_caption_dict[train_image_ids[0]]:')
pprint(train_caption_dict[train_image_ids[0]])

# of images in train set: 6000
# of features in train set: 6000
# of images in train caption dict: 6000
# of images in valid set: 1000
# of features in valid set: 1000
# of images in valid caption dict: 1000
-------------------------------------------------------------------
Exmaples in each variable
train_image_ids[0]: 2513260012_03d33305cf
train_feature_dict[train_image_ids[0]].shape: (1, 4096)
train_caption_dict[train_image_ids[0]]:
['startseq black dog is running after white dog in the snow endseq',
 'startseq black dog chasing brown dog through snow endseq',
 'startseq two dogs chase each other across the snowy ground endseq',
 'startseq two dogs play together in the snow endseq',
 'startseq two dogs running through low lying body of water endseq']


In [6]:
def get_all_captions(captions_dict):
    result = []
    for image_id in captions_dict:
        caption_list = captions_dict[image_id]
        [result.append(x) for x in caption_list]
    
    return result

In [7]:
def get_tokenizer(all_captions):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(all_captions)
    vocab_size = len(tokenizer.word_index) + 1 # cf) https://github.com/keras-team/keras/issues/7551
    return tokenizer, vocab_size

In [8]:
tokenizer, vocab_size = get_tokenizer(get_all_captions(train_caption_dict))

print('len all train captions:', len(get_all_captions(train_caption_dict)))

word_set = set()
for caption in get_all_captions(train_caption_dict):
    caption = caption.split()[:]
    word_set.update(caption)
    
print('# of distinct word in all captions:', len(word_set))
print('vocab_size:', vocab_size)
print(set(tokenizer.word_docs.keys()) == word_set) # True

len all train captions: 30000
# of distinct word in all captions: 7578
vocab_size: 7579
True


In [9]:
# get maximun length of caption
def get_max_length(all_captions):
    length = 0
    for caption in all_captions:
        if len(caption.split()) > length:
            length = len(caption.split())
            
    return length

In [10]:
'''
our model receive 2 inputs: image feature vector, sequence of words
our model outputs 1 word
if given image feature and caption: 'there is dog', we reuse subsequence of caption
image, 'startseq' => 'there'
image, 'startseq', 'there' => 'is'
image, 'startseq', 'there', 'is' => 'dog'
image, 'startseq', 'there', 'is', 'dog' => 'endseq'

tokenizer: tokenizer from train set
maxlen: maximum length of captions in train set
vocab_size: vocab_size extracted from train set
image_ids: image ids of data set which will be fed into model(train, valid, test)
feature_dict: feature dict of data set which will be fed into model(train, valid, test)
caption_dict: caption dict of data set which will be fed into model(train, valid, test)
'''

def get_subsequence_inputs(tokenizer, maxlen, vocab_size, image_ids, feature_dict, caption_dict):
    image, subsequence, output = list(), list(), list()
    for image_id in image_ids:
        captions = caption_dict[image_id]
        for caption in captions:
            # convert 'word sequnce' to 'int sequence'
            # each int implies word in vocab
            vector = tokenizer.texts_to_sequences([caption])[0] 
            for i in range(1, len(vector)):
                subseq, out = vector[:i], vector[i]
                # add padding to make input sequence has same fixed length
                subseq = pad_sequences([subseq], maxlen=maxlen)[0]
                # one hot encoding
                out = to_categorical([out], num_classes=vocab_size)[0]
                image.append(feature_dict[image_id][0]) # image feature vector
                subsequence.append(subseq) # vetorized subsequence of fixed size
                output.append(out) # one-hot encoded word vector
                
    return np.array(image), np.array(subsequence), np.array(output)

In [11]:

# for image_id, caption_list in train_captions.items():
#     for caption in caption_list:
#         vec = tokenizer.texts_to_sequences([caption])[0] # convert to vector such that mapping each word to int based on vocab_size
#         for i in range(1, len(vec)):
#             seq_in, seq_out = vec[:i], vec[i] # split into pair
#             seq_in = pad_sequences([seq_in], maxlen=max_caption_length)[0] # add padding to make all caption vector has same length
#             seq_out = to_categorical([seq_out], num_classes=vocab_size)[0] # one hot encoding
#             x1.append(train_features[image_id][0]) # feature vector
#             x2.append(seq_in) # input word sequence to fixed length int vector: max length of input caption
#             y.append(seq_out) # output word to one-hot encoded vector of fixed length: vocab_size


In [12]:
start = time.time()
maxlen = get_max_length(get_all_captions(train_caption_dict))
x1_train, x2_train, y_train = get_subsequence_inputs(tokenizer, maxlen, vocab_size,
                                                     train_image_ids, train_feature_dict, train_caption_dict)            
x1_valid, x2_valid, y_valid = get_subsequence_inputs(tokenizer, maxlen, vocab_size,
                                                     valid_image_ids, valid_feature_dict, valid_caption_dict)            

end = time.time()

In [13]:
print('Elapsed time:', end-start, 'sec')
print('maxlen:', maxlen)
print('total # of inputs to model:', len(x1_train))
print('Example of subsequnce inputs below')
print('x1_train:', x1_train[2].shape) # image feature vector
pprint(x1_train[2])
print('x2_train:', x2_train[2].shape) # vectorized fixed length word sequence
pprint(x2_train[2])
print('y_train:',y_train[2].shape) # one hot encoded word vector
pprint(y_train[2])

Elapsed time: 57.90935921669006 sec
maxlen: 34
total # of inputs to model: 306404
Example of subsequnce inputs below
x1_train: (4096,)
array([2.5076475, 0.       , 0.       , ..., 0.       , 0.       ,
       0.       ], dtype=float32)
x2_train: (34,)
array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, 14,  8],
      dtype=int32)
y_train: (7579,)
array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)


In [14]:
# define model
# vocab_size, maxlen already described above
def get_model(vocab_size, maxlen):
    # image
    inputs1 = Input(shape=(4096,)) # feature extracted from pretrained VGG16
    image = Dropout(0.5)(inputs1) # dropout layer
    image = Dense(256, activation='relu')(image) # fc layer outputs 256 features
    
    # caption
    inputs2 = Input(shape=(maxlen,)) # vectorized word sequence
    # word embedding base on 'vocab_size' to make simliar word has similar vector
    # shape: (maxlen) -> (maxlen, 256)
    # 'make_zero=True' option will ignore padded word
    caption = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    caption = Dropout(0.5)(caption) # dropout layer
    caption = LSTM(256)(caption) # LSTM layer
    
    # decoder
    decoder = add([image, caption])
    decoder = Dense(256, activation='relu')(decoder) # fc layer
    outputs = Dense(vocab_size, activation='softmax')(decoder) # probability of each vocab
    
    # make model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam') # set loss function and optimizer
    
    # visualize model summary
    print(model.summary())

    return model

In [16]:
# possible to use this cell when you have enough memory
'''
# model checkpoint
start = time.time()

filepath = 'model-epoch{epoch:03d}-loss{loss:.3f}-validset_loss{val_loss:.3f}.h5' # format string
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# fit model
model = get_model(vocab_size, maxlen)
print('x1_train:', x1_train[2].shape) # image feature vector
print('x2_train:', x2_train[2].shape) # vectorized fixed length word sequence
print('y_train:',y_train[2].shape) # one hot encoded word vector

print('x1_valid:', x1_valid[2].shape) # image feature vector
print('x2_valid:', x2_valid[2].shape) # vectorized fixed length word sequence
print('y_valid:',y_valid[2].shape) # one hot encoded word vector


model.fit([x1_train, x2_train], y_train, epochs=1, verbose=2, callbacks=[checkpoint], validation_data=([x1_valid, x2_valid], y_valid))

end = time.time()
print('Elapsed:', end-start, 'sec')
'''

"\n# model checkpoint\nstart = time.time()\n\nfilepath = 'model-epoch{epoch:03d}-loss{loss:.3f}-validset_loss{val_loss:.3f}.h5' # format string\ncheckpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')\n\n# fit model\nmodel = get_model(vocab_size, maxlen)\nprint('x1_train:', x1_train[2].shape) # image feature vector\nprint('x2_train:', x2_train[2].shape) # vectorized fixed length word sequence\nprint('y_train:',y_train[2].shape) # one hot encoded word vector\n\nprint('x1_valid:', x1_valid[2].shape) # image feature vector\nprint('x2_valid:', x2_valid[2].shape) # vectorized fixed length word sequence\nprint('y_valid:',y_valid[2].shape) # one hot encoded word vector\n\n\nmodel.fit([x1_train, x2_train], y_train, epochs=1, verbose=2, callbacks=[checkpoint], validation_data=([x1_valid, x2_valid], y_valid))\n\nend = time.time()\nprint('Elapsed:', end-start, 'sec')\n"

In [140]:
# data generator to fit in workstation memory
def data_generator(caption_dict, feature_dict, tokenizer, maxlen):
    while True:
        for image_id, caption_list in caption_dict.items():
            image_in = feature_dict[image_id][0]
            image_in, seq_in, out_word = get_subsequence_inputs(tokenizer, maxlen, caption_list, image_in)
            yield [[image_in, seq_in], out_word]
            
def get_subsequence_inputs(tokenizer, maxlen, caption_list, image_in):
    x1, x2, y = list(), list(), list()
    vocab_size = len(tokenizer.word_index) + 1
    
    for caption in caption_list:
        print('caption:', caption)
        seq = tokenizer.texts_to_sequences([caption])[0]
        
        # make subsequence inputs
        for i in range(1, len(seq)):
            seq_in, out_word = seq[:i], seq[i]
            print('out_word:', out_word)
            seq_in = pad_sequences([seq_in], maxlen=maxlen)[0]
            out_word = to_categorical([out_word], num_classes=vocab_size)[0]
            x1.append(image_in)
            x2.append(seq_in)
            y.append(out_word)
            
    return np.array(x1), np.array(x2), np.array(y)

In [133]:
generator = data_generator(train_caption_dict, train_feature_dict, tokenizer, maxlen)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)


(45, 4096)
(45, 34)
(45, 7579)


In [141]:
# train using generator for saving memory
model = get_model(vocab_size, maxlen)
epochs = 1
steps = len(train_caption_dict)
start = time.time()
for i in range(epochs):
    generator = data_generator(train_caption_dict, train_feature_dict, tokenizer, maxlen)
    for i in range(5):
        inputs, outputs = next(generator)
        print('----------------------------')
#     model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
#     model.save('model_{}.h5'.format(i))
    
end = time.time()
print('Elaspsed:', end-start, 'sec')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 34)           0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 34, 256)      1940224     input_12[0][0]                   
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 4096)         0           input_11[0][0]                   
__________________________________________________________________________________________________
dropout_11

In [63]:
# genetrating caption
idx_to_word = dict()
for word in tokenizer.word_index:
    idx = tokenizer.word_index[word]
    idx_to_word[idx] = word
    
    
# given image feature and model, generate caption for image
# 'startseq' and image feature will be fed into model
# create each word until 'endseq' or reach maxlen
def generate_caption(model, tokenizer, image_feature, maxlen):
    seq_in = 'startseq'
    
    for i in range(maxlen):
        seq_vector = tokenizer.texts_to_sequences([seq_in])[0]
        seq_vector = pad_sequences([seq_vector], maxlen=maxlen) # add padding to sequence vector
        pred_word = model.predict([image_feature, seq_vector], verbose=2) # predict next word
        pred_word = np.argmax(pred_word) # most probable word's index
        word = idx_to_word.get(pred_word, None)

        # may be error case
        if word is None:
            print('index not in idx_to_word:', pred_word)
            break
            
        if word == 'endseq':
            seq_in += ' ' + word
            break
            
        seq_in += ' ' + word

    return seq_in

In [139]:
print(idx_to_word[15])
print(idx_to_word[54])
print(idx_to_word[39])

boy
group
little


In [110]:
# evaluate performance
def evaluate_model(model, y_caption_dict, y_feature_dict, tokenizer, maxlen):
    y, pred = list(), list()
    for image_id, caption_list in y_caption_dict.items():
        _pred = generate_caption(model, tokenizer, y_feature_dict[image_id], maxlen) # generate caption
        y.append([c.split() for c in caption_list]) # collect answer captions
        pred.append(_pred.split()) # collect prediction
        print(image_id, _pred)
        
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(y, pred, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(y, pred, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(y, pred, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(y, pred, weights=(0.25, 0.25, 0.25, 0.25)))

In [108]:
# prepare test set
test_info_filename = os.getcwd() + '/Flickr8k_text/Flickr_8k.testImages.txt'
test_image_ids, test_feature_dict, test_caption_dict = prepare_dataset(test_info_filename)
print('# of images in test set:', len(test_image_ids))
print('# of features in test set:', len(test_feature_dict)) # this should be same with above


# of images in test set: 1000
# of features in test set: 1000


In [114]:
# load model and evaluate
start = time.time()

model_filename = 'model_0.h5' # 1 epoch
model1 = load_model(model_filename)
evaluate_model(model1, test_caption_dict, test_feature_dict, tokenizer, maxlen)

end = time.time()
print('Elapsed:', end-start, 'sec')

506738508_327efdf9c3 startseq man in red shirt is standing on the street endseq
1982852140_56425fa7a2 startseq man in red shirt is standing on the street endseq
3232470286_903a61ea16 startseq man in red shirt is standing on the street endseq
2844641033_dab3715a99 startseq man in red shirt is standing on the street endseq
3245070961_8977fdd548 startseq man in red shirt is standing on the street endseq
293327462_20dee0de56 startseq man in red shirt is standing on the street endseq
476759700_8911f087f8 startseq man in red shirt is standing on the street endseq
3406930103_4db7b4dde0 startseq man in red shirt is standing on the street endseq
2602258549_7401a3cdae startseq man in red shirt is standing on the street endseq
3354330935_de75be9d2f startseq man in red shirt is standing on the street endseq
1626754053_81126b67b6 startseq man in red shirt is standing on the street endseq
670609997_5c7fdb3f0b startseq man in red shirt is standing on the street endseq
3562050678_4196a7fff3 startseq m

KeyboardInterrupt: 

In [113]:
# load model and evaluate
start = time.time()

model_filename = 'model_19.h5' # 20 epochs
model2 = load_model(model_filename)
evaluate_model(model2, test_caption_dict, test_feature_dict, tokenizer, maxlen)

end = time.time()
print('Elapsed:', end-start, 'sec')

506738508_327efdf9c3 startseq man in black shirt and hat is sitting on bench endseq
1982852140_56425fa7a2 startseq man in black shirt and hat is sitting on bench endseq
3232470286_903a61ea16 startseq man in black shirt and hat is sitting on bench endseq
2844641033_dab3715a99 startseq man in black shirt and hat is sitting on bench endseq
3245070961_8977fdd548 startseq man in black shirt and hat is sitting on bench endseq
293327462_20dee0de56 startseq man in black shirt and hat is sitting on bench endseq
476759700_8911f087f8 startseq man in black shirt and hat is sitting on bench endseq
3406930103_4db7b4dde0 startseq man in black shirt and hat is sitting on bench endseq
2602258549_7401a3cdae startseq man in black shirt and hat is sitting on bench endseq
3354330935_de75be9d2f startseq man in black shirt and hat is sitting on bench endseq
1626754053_81126b67b6 startseq man in black shirt and hat is sitting on bench endseq
670609997_5c7fdb3f0b startseq man in black shirt and hat is sitting 

KeyboardInterrupt: 

In [ ]:
# catpion generation example on test dataset
start = time.time()

max_example = [0, 0, 0] # [bleu_score, image_id, prediction]
min_example = [1, 0, 0] # [bleu_score, image_id, prediction]
for image_id in test_image_ids:
    y = test_caption_dict[image_id]
    y = [[x.split() for x in y]]
    pred = generate_caption(model1, tokenizer, test_feature_dict[image_id], maxlen)
    pred = [pred.split()]
    bleu_score = corpus_bleu(y, pred, weights=(1.0, 0, 0, 0))
    
    if bleu_score > max_example[0]:
        max_example[0] = bleu_score
        max_example[1] = image_id
        max_example[2] = pred
        
    if bleu_score < min_example[0]:
        min_example[0] = bleu_score
        min_example[1] = image_id
        min_example[2] = pred
    
end = time.time()
print('Elapsed:', end-start, 'sec')

In [ ]:
print('max bleu score:', max_example[0])
image_filename = './Flicker8k_Dataset/' + max_example[1] + '.jpg'
print(image_filename)
for c in test_caption_dict[max_example[1]]:
    print(' '.join(c.split()[1:-1]))

display(Image(filename=image_filename))

print(' '.join(max_example[2][0][1:-1]))

In [ ]:
print('min bleu score:', min_example[0])
image_filename = './Flicker8k_Dataset/' + min_example[1] + '.jpg'
print(image_filename)
for c in test_caption_dict[min_example[1]]:
    print(' '.join(c.split()[1:-1]))

display(Image(filename=image_filename))

print(' '.join(min_example[2][0][1:-1]))